# Homework 1
## Question 7
This requires we implement the Perceptron Learning Algorithm (PLA) for target functions $f: \mathcal{X} \rightarrow \mathcal{Y}$, where $\mathcal{X} = [-1, 1]^2$ and $\mathcal{Y} = \{-1, +1\}$, that we assume are linearly seperable. This means that the target function is determined by a line in $\mathcal{X} = [-1, 1] \times [-1, 1]$ and all points on one side of the line are assigned either $+1$ or $-1$ and all points on the other side of the line are assigned the other.